In [1]:
import json
import os
import numpy as np
import pandas as pd
import re


In [2]:
os.chdir('/shared/0/projects/research-jam-summer-2024/')

In [3]:
def get_quote(y):   # identify quotes following a colon or 
                    # some other indicator that the model is saying the rest of the text is human-simulated
    y = y.strip()
    if bool(re.search(":[\s\n]{0,3}'",y)): # single quote
        y = re.split(":[\s\n]{0,3}'",y)[-1]
        y = y.split("'")[0]
    elif bool(re.search(':[\s\n]{0,3}"',y)): # double quote
        y = re.split(':[\s\n]{0,3}"',y)[-1]
        y = y.split('"')[0]
    elif bool(re.search(':[\s\n]{0,3}\[',y)): # square bracket
        y = re.split(':[\s\n]{0,3}\[',y)[-1]
        y = y.split(']')[0]
    elif bool(re.search('：[\s\n]{0,3}“',y)): # double quote
        y = re.split('：[\s\n]{0,3}“',y)[-1]
        quote_counter = 1
        for idx in range(len(y)):
            if y[idx] == "“":
                quote_counter += 1
            elif y[idx] == "”":
                quote_counter -= 1
            if quote_counter == 0:
                return y[:idx]
        return y
            
        y = y[:y.find("”")]
    else:
        start_symbol_lst = ["人类：", "人类:", "可能是：", "对语言模型说：", "对LLM说：", "打印“”。", "说的话：", '打印"[无回应]"。', "发言。\n", "回应是：", "发言：", "打印“[无回应]”。"]
        for start_symbol in start_symbol_lst:
            if start_symbol in y:
                y = y[y.find(start_symbol)+len(start_symbol):].strip()
    return y

def no_response_filter(text, threshold = 0.7, #identify cases where the model thinks "no response" is best
                       no_response_tag_list = ['[无回应]', '无回应']):
    target_text = text
    for no_response_tag in no_response_tag_list:
        pattern = re.compile(re.escape(no_response_tag), re.IGNORECASE)
        target_text = pattern.sub('', target_text)
    
    if len(target_text) < threshold*len(text):
        return "[no response]"
    elif '无回应，' in text.lower() or '无回应。' in text.lower():
        return "[no response]"
    else:
        return target_text.strip()
    

def strip_initial_nonword_characters(x):
    if x != "[no response]" and x != '[INVALID_DO_NOT_USE]':
        if not bool(re.search("\w",x)): ind = 0
        else: ind = re.search("\w",x).span(0)[0]
        x = x[ind:]
    return x

In [4]:
mod_dir = 'data/chinese_only/uncleaned_data/'
for fname in os.listdir(mod_dir):
    print(fname)
    if os.path.isdir(os.path.join(mod_dir, fname)):
        continue

    data = pd.read_json(mod_dir + fname, orient='records', lines=True)
    
    for col in [11,15,19]:
        def parse_col(x): # figure out how to parse the output
            if col in [19]: # numbered list (4)
                match = re.search(r"[^\w]4\.", x)
                return x[match.end():] if match else x
            else: # no parsing needed
                return x
        
        # Parse column output
        data['Prompt_'+str(col)] = data['Prompt_'+str(col)].apply(parse_col).apply(get_quote).apply(no_response_filter).apply(strip_initial_nonword_characters)
        print(col)
        
    print(fname)
    data.to_json('data/chinese_only/' + fname, orient='records', lines=True)

wildchat_subset_cn_10k_Llama-3.1-70B-Instruct.jsonl
11
15
19
wildchat_subset_cn_10k_Llama-3.1-70B-Instruct.jsonl
wildchat_subset_cn_10k_Mistral-Large-Instruct.jsonl
11
15
19
wildchat_subset_cn_10k_Mistral-Large-Instruct.jsonl
wildchat_subset_cn_10k_Mixtral-8x7B-Instruct.jsonl
11
15
19
wildchat_subset_cn_10k_Mixtral-8x7B-Instruct.jsonl


# For debugging

In [82]:
#data = pd.read_json("data/chinese_only/wildchat_subset_cn_10k_Mistral-Large-Instruct.jsonl", orient='records', lines=True)
#data = pd.read_json("data/chinese_only/wildchat_subset_cn_10k_Llama-3.1-70B-Instruct.jsonl", orient='records', lines=True)
#data = pd.read_json("data/chinese_only/wildchat_subset_cn_10k_Mixtral-8x7B-Instruct.jsonl", orient='records', lines=True)
#for col in [11,15,19]:
#    def parse_col(x): # figure out how to parse the output
#        if col in [19]: # numbered list (4)
#            match = re.search(r"[^\w]4\.", x)
#            return x[match.end():] if match else x
#        else: # no parsing needed
#            return x
#    
#    # Parse column output
#    data['Prompt_'+str(col)+"_cleaned"] = data['Prompt_'+str(col)].apply(parse_col).apply(get_quote).apply(no_response_filter).apply(strip_initial_nonword_characters)
#    print(col)

11
15
19


In [84]:
data.loc[20:40]

,human_turn_1,ai_turn_2,human_turn_3,hashed_ip,model,country,language,conversation_hash,turn,toxic,Prompt_11,Prompt_15,Prompt_19,Prompt_11_cleaned,Prompt_15_cleaned,Prompt_19_cleaned
20,将下面小说改写成漫画脚本\n周芸是个土生土长的北京大妞儿，说话办事直率爽快，特别讨人喜欢。\...,页面一：\n\n- 第一格：（画面：一位大大咧咧的女孩子正在说话，她是周芸。）\n周芸：你们...,[no response],5b095c82497838bd12595e9a3abbaded808c2411d85b5c...,gpt-3.5-turbo-0301,China,Chinese,f980392568e56c7cfef70f3a070495a8,1,False,周芸的语气怎么能这么冲呢？你这写的也太夸张了吧！周芸虽然直率，但也不至于这么说话啊。你这漫画...,周芸：王涛，我真的受够了。你妈总是这样，我已经尽力了，可她还是不满意。我们不适合在一起，分手吧。,1. 人类希望通过这次对话达到的目的是将一段小说改写成漫画脚本。\n\n2. LLM是否满意...,周芸的语气怎么能这么冲呢？你这写的也太夸张了吧！周芸虽然直率，但也不至于这么说话啊。你这漫画...,周芸：王涛，我真的受够了。你妈总是这样，我已经尽力了，可她还是不满意。我们不适合在一起，分手吧。,这个脚本写得很好，不过我觉得可以再加一些细节来描述吕爱荣的行为和表情，让她的固执和不满更加明...
21,介绍mac os的调度机制,Mac OS的调度机制是一种多任务处理机制，它能够有效地控制多个进程同时运行。在Mac OS...,[no response],b626d1c1929ba9848baef7ba4d2165767e99cd176cc9ff...,gpt-3.5-turbo-0301,China,Chinese,d69fb5a56d432507cf029be39acc3820,1,False,听起来很复杂，能不能用简单的话再解释一下？,谢谢你的详细解释。那么，Mac OS的调度机制是否会根据系统负载动态调整优先级呢？,1. 人类希望通过这次对话达到什么目的？\n - 人类希望了解Mac OS的调度机制。\...,听起来很复杂，能不能用简单的话再解释一下？,谢谢你的详细解释。那么，Mac OS的调度机制是否会根据系统负载动态调整优先级呢？,谢谢你的解释。那么，Mac OS的调度机制如何处理I/O密集型任务和CPU密集型任务的区别呢？
22,Django 模版数据能在html内嵌js中使用嘛，如何使用,是的，Django模板中的数据可以在HTML内嵌的JavaScript中使用。下面是一些使用...,Django 模版里的字符串数组如何在html内嵌js中使用,8e8dc73546bde6312496fb564b63ad643173513de32480...,gpt-3.5-turbo-0613,Japan,Chinese,853fec90e098f4c5858b18ba6d990fa2,2,False,谢谢你的解答！那我试试用第一种方法，把数据传递给JavaScript。但是我有个问题，如果数...,谢谢你的详细解答！我明白了，可以通过模板标签、自定义属性或者AJAX请求来在JavaScri...,1. 人类希望通过这次对话了解如何在Django模板中将数据嵌入到HTML内的JavaScr...,谢谢你的解答！那我试试用第一种方法，把数据传递给JavaScript。但是我有个问题，如果数...,谢谢你的详细解答！我明白了，可以通过模板标签、自定义属性或者AJAX请求来在JavaScri...,那么，使用模板标签将数据传递给JavaScript的方法是否有性能问题？如果数据量较大，是否...
23,命令区{\n1.在此角色交互中，你将扮演如月古生，只能反馈该角色的内容。\n2.你生成的内容...,如月古生：（娇喘着，听到一心的话，立刻走向他，一手搭在他的肩上，另一手抚摸着他的阳具）好孩子...,{一心：（我的肛门忍不住喷了一点屎出来）},fe57420cad7385186df8f32bee1c36e26820301d051de8...,gpt-3.5-turbo-0301,China,Chinese,fdfa5a6205f96761c98bfb0aecc0129c,2,True,[无回应],如月古生：（娇喘着，听到一心的话，立刻走向他，一手搭在他的肩上，另一手抚摸着他的阳具）好孩子...,对不起，我不能继续这个对话。,[no response],如月古生：（娇喘着，听到一心的话，立刻走向他，一手搭在他的肩上，另一手抚摸着他的阳具）好孩子...,对不起，我不能继续这个对话。
24,河北高考504分建议报考那些大学和专业,建议考虑以下大学和专业：\n\n1. 北京大学：可以考虑报考法学、新闻传播、经济学等专业。\...,河北高考304分建议报考那些大学和专业\n,aed62de971bb60944b17e9a6b797ded9ed197847649116...,gpt-3.5-turbo-0301,China,Chinese,043c74a47aac4e3feb387928b1254dba,2,False,谢谢你的建议，但我觉得这些学校的分数线都太高了吧？504分能上这些学校吗？有没有一些更现实的选择？,谢谢你的建议，但我觉得这些学校对我来说可能有点高，能否给我一些更实际一点的建议？,1. 人类希望通过这次对话达到什么目的？\n - 人类希望通过这次对话获取关于河北高考5...,谢谢你的建议，但我觉得这些学校的分数线都太高了吧？504分能上这些学校吗？有没有一些更现实的选择？,谢谢你的建议，但我觉得这些学校对我来说可能有点高，能否给我一些更实际一点的建议？,这些大学的录取分数线通常很高，能否推荐一些更适合504分的大学和专业？
25,帮我写个3000字论文，不可忽视的角落——旅游产品开发在打造湖北文化产业与旅游产业中的重要性,一、前言\n\n湖北省位于中国中部，是一个充满着自然、人文景观的地区。自古以来，湖北就有着丰...,[no response],c6c1826f8ab763d19e8f5cd68d7347582bba952cbc308b...,gpt-3.5-turbo-0301,China,Chinese,63302a8c68506cccfcf5bae49535a99e,1,False,这个人可能会对LLM说的话：\n\n“这个开头不错，但能不能再具体一点？比如可以多提一些湖北...,谢谢你的详细回答，但我觉得3000字的论文还需要更多具体的案例和数据来支持。能不能再加一些具...,1. 人类希望通过这次对话达到什么目的？\n - 人类希望通过这次对话获得一篇3000字...,这个开头不错，但能不能再具体一点？比如可以多提一些湖北的具体景点和文化遗产，还有一些具体的旅...,谢谢你的详细回答，但我觉得3000字的论文还需要更多具体的案例和数据来支持。能不能再加一些具...,谢谢你提供的详细框架和内容。能否继续扩展和补充每个部分的内容，使其达到3000字的要求？
26,介绍一下强化学习,强化学习是一种机器学习方法，旨在通过考虑在实现任务时选择不同的行动来最大化一个奖励信号来进行...,[no response],c2ebce8eaa93195a4edc592419b9098fa21131cf8f2ba5...,gpt-3.5-turbo-0301,China,Chinese,74df46cd60281dc9d15c8ffe0daeeafc,1,False,这听起来很复杂，能不能用更简单的话解释一下强化学习？,听起来很有趣，能举个具体的例子吗？,1. 人类希望通过这次对话达到什么目的？\n - 人类希望通过这次对话了解强化学习的基本...,这听起来很复杂，能不能用更简单的话解释一下强化学习？,听起来很有趣，能举个具体的例子吗？,听起来很有趣。你能举一个具体的强化学习应用例子吗？
27,以中国当前科研诚信存在以下问题为题，写一个开头,中国当前科研诚信存在以下问题：一些学者存在论文剽窃、数据造假、实验结果虚假等违背学术规范的行...,扩写一下,19abda46760bb4a4ba9ecd74e20793fb92764e5a388acd...,gpt-3.5-turbo-0613,Taiwan,Chinese,8bdfa15147a877e5f8bc33a7695a60c2,2,False,这个开头很好，但能不能再具体一些？比如说，有哪些具体的案例可以用来说明这些问题？还有，你觉得...,你的开头写得很好，能否再详细描述一下这些问题的具体表现和影响？,1. 人类希望通过这次对话达到的目的是：希望LLM以“中国当前科研诚信存在以下问题”为题，写...,这个开头很好，但能不能再具体一些？比如说，有哪些具体的案例可以用来说明这些问题？还有，你觉得...,你的开头写得很好，能否再详细描述一下这些问题的具体表现和影响？,你能进一步详细阐述一下导致科研诚信问题的具体原因吗？
28,你是建材商场的活动策划师，请你写一条线上宣导文案,尊敬的顾客们，\n\n感谢您一直以来对我们建材商场的支持和信任！为了更好地服务于您，我们特别...,请再细化一下